In [3]:
# import library
import pandas as pd

In [4]:
#import dataset
import requests
from io import StringIO
 
orig_url='https://drive.google.com/file/d/1_caTB8v0bxwOttXQhgXxhBfcrgH4G4oo/view?usp=sharing'
 
file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
df = pd.read_csv(csv_raw)
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [5]:
# first 5 data
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [6]:
# last 5 data
data = df[['class','tweet']]
data.tail(5)

,class,tweet
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies
24782,2,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...


In [7]:
# split tweet and class
twit = data['tweet'].values


# check stop words
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
print(stopwords.words('english'))

# remove stop words
import nltk
nltk.download('punkt')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

twit = str(data['tweet'].values)

stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(twit) 

new_twit = [w for w in word_tokens if not w in stop_words] 

new_twit = [] 

for w in word_tokens: 
	if w not in stop_words: 
		new_twit.append(w) 

print(word_tokens) 
print(new_twit) 


In [8]:
# split class and tweet
kelas = data['class'].values


In [9]:
# import tensorflow
# create categorical data
import tensorflow as tf
kelas = tf.keras.utils.to_categorical(kelas, 3)
data['class'].unique()

array([2, 1, 0])

In [10]:
# check each tweet is text
for i in range(len(twit)):
  twit[i] = str(twit[i])

In [11]:
# split train and test data for tweet and class
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(twit, kelas, test_size=0.2)

In [12]:
# add tokenizer and pad_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
# process tokenization
MAX_NB_WORDS =1000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_BIN = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',lower=True)
tokenizer.fit_on_texts(data['tweet'].values)
word_index = tokenizer.word_index

x = tokenizer.texts_to_sequences(df['tweet'].values)
x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)


In [26]:
# process tokenization
tokenizer = Tokenizer(num_words=1000, oov_token='x')
tokenizer.fit_on_texts(x_train) 


# The FastText Algorithm  for OOV

In [27]:
# convert sample into sequence
seq_train = tokenizer.texts_to_sequences(x_train)
seq_test = tokenizer.texts_to_sequences(x_test)


In [28]:
# add padding train and test data
pad_train = pad_sequences(seq_train) 
pad_test = pad_sequences(seq_test)


In [32]:
# create some layers model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_BIN, input_length=x.shape[1]),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [33]:
# compiling model
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError()])

In [34]:
# training model
model.fit(pad_train, y_train, epochs=10, batch_size=64, validation_data=(pad_test, y_test), verbose=2)

Epoch 1/10


310/310 - 23s - loss: 0.6946 - accuracy: 0.7715 - mean_absolute_error: 0.2705 - val_loss: 0.6482 - val_accuracy: 0.7856 - val_mean_absolute_error: 0.2481
Epoch 2/10
310/310 - 22s - loss: 0.6666 - accuracy: 0.7715 - mean_absolute_error: 0.2486 - val_loss: 0.6468 - val_accuracy: 0.7856 - val_mean_absolute_error: 0.2438
Epoch 3/10
310/310 - 23s - loss: 0.6666 - accuracy: 0.7715 - mean_absolute_error: 0.2481 - val_loss: 0.6468 - val_accuracy: 0.7856 - val_mean_absolute_error: 0.2433
Epoch 4/10
310/310 - 23s - loss: 0.6650 - accuracy: 0.7715 - mean_absolute_error: 0.2479 - val_loss: 0.6354 - val_accuracy: 0.7856 - val_mean_absolute_error: 0.2461
Epoch 5/10
310/310 - 23s - loss: 0.5095 - accuracy: 0.8212 - mean_absolute_error: 0.1942 - val_loss: 0.3508 - val_accuracy: 0.8842 - val_mean_absolute_error: 0.1309
Epoch 6/10
310/310 - 23s - loss: 0.3233 - accuracy: 0.8963 - mean_absolute_error: 0.1159 - val_loss: 0.3016 - val_accuracy: 0.8971 - val_mean_absolute_error: 0.1085
Epoch 7/10
310/310 - 